In [1]:
!pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=c66874ceb686d447ff5e36b36112c97a5e69ed0b9ad0e0d7b9052f67b7765de1
  Stored in directory: /root/.cache/pip/wheels/6f/db/e2/945dc8c17477cf9ab56d9f9aa1fa75b75335fffff9b12d867d
Successfully built spark


#PERGUNTAS SOBRE:

Há inconsistencias nos dados?

É possivel identificar caractericas de sobrepeso de dados, já que este atributo não existe?

Qual o resultado é possivel interpretar com a pesquisa do sobrepeso?

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Foodf").getOrCreate()

In [10]:
df = spark.read.csv("food_coded.csv", header=True, inferSchema=True)

In [11]:
df.show(10)

+---------------+------+--------------------+----------------+------------+--------------+------+--------------------+--------------------+---------------------------+----+----------------------------+-------+--------------------+------------------+-----+--------------------+--------------------+---------------------+----------+----------+-----------+--------+----------------+-----------------+--------------------+-----------------+--------+--------------------+-----+---------+-----------+----------+---------------+--------------------+--------------------+----------------+------+-----------+------------+--------------+--------------+--------------------+----------------+--------------------+-----------------+-------------+------------+------------+------------+----------------------+----+------+---------+-----------------+---------------+-----------+-----------+--------+---------------+--------------------+
|            GPA|Gender|           breakfast|calories_chicken|calories_day|cal

In [13]:
df.select('mother_profession','type_sports','Gender','weight','calories_day').show(30)

+--------------------+--------------------+------+--------------------+------------+
|   mother_profession|         type_sports|Gender|              weight|calories_day|
+--------------------+--------------------+------+--------------------+------------+
|          unemployed|          car racing|     2|                 187|         nan|
|           Nurse RN |         Basketball |     1|                 155|           3|
|       owns business|                none|     1|I'm not answering...|           4|
|                NULL|                NULL|     1|                NULL|           3|
|                NULL|                NULL|  NULL|                NULL|        NULL|
|                NULL|                NULL|     4|                NULL|           1|
|Substance Abuse C...|            Softball|     1|                 190|           2|
|        Hair Braider|               None.|     1|                 190|           3|
|          Journalist|              soccer|     2|               

In [14]:
from pyspark.sql.functions import *
resultdf1 = df.filter(df.weight.isNotNull()).select('mother_profession','type_sports','Gender','weight','calories_day')

In [15]:
resultdf1.show()

+--------------------+--------------------+------+--------------------+------------+
|   mother_profession|         type_sports|Gender|              weight|calories_day|
+--------------------+--------------------+------+--------------------+------------+
|          unemployed|          car racing|     2|                 187|         nan|
|           Nurse RN |         Basketball |     1|                 155|           3|
|       owns business|                none|     1|I'm not answering...|           4|
|Substance Abuse C...|            Softball|     1|                 190|           2|
|        Hair Braider|               None.|     1|                 190|           3|
|          Journalist|              soccer|     2|                 180|           3|
|                cook|                none|     1|                 137|           3|
|Elementary School...|                none|     1|                 180|         nan|
|  Pharmaceutical rep|        field hockey|     1|               

In [19]:
resultdf2 = resultdf1.filter(col('weight').rlike('^[0-9]*$'))
resultdf2.show()

+--------------------+--------------------+------+------+------------+
|   mother_profession|         type_sports|Gender|weight|calories_day|
+--------------------+--------------------+------+------+------------+
|          unemployed|          car racing|     2|   187|         nan|
|           Nurse RN |         Basketball |     1|   155|           3|
|Substance Abuse C...|            Softball|     1|   190|           2|
|        Hair Braider|               None.|     1|   190|           3|
|          Journalist|              soccer|     2|   180|           3|
|                cook|                none|     1|   137|           3|
|Elementary School...|                none|     1|   180|         nan|
|  Pharmaceutical rep|        field hockey|     1|   125|           3|
|     Chidos Cleaners|              soccer|     1|   116|           3|
|     Court Reporter |             Running|     1|   110|           4|
|Child care provider |Soccer and basket...|     2|   264|           3|
|     

In [20]:
resultdf2.groupBy('Gender','weight').count().orderBy('weight').show()

+------+------+-----+
|Gender|weight|count|
+------+------+-----+
|     1|     1|    1|
|     1|   100|    1|
|     1|   105|    1|
|     1|   110|    1|
|     1|   112|    1|
|     1|   113|    1|
|     1|   115|    1|
|     1|   116|    1|
|     1|   120|    3|
|     1|   123|    1|
|     1|   125|    4|
|     1|   127|    1|
|     1|   128|    2|
|     1|   129|    2|
|     1|   130|    3|
|     1|   135|    7|
|     2|   135|    1|
|     1|   137|    1|
|     2|   140|    2|
|     1|   140|    6|
+------+------+-----+
only showing top 20 rows



In [21]:
resultdf3 = resultdf2.select('type_sports', when(resultdf2.weight >= 150, 'Sobrepeso'))

In [22]:
resultdf3.show()

+--------------------+--------------------------------------------+
|         type_sports|CASE WHEN (weight >= 150) THEN Sobrepeso END|
+--------------------+--------------------------------------------+
|          car racing|                                   Sobrepeso|
|         Basketball |                                   Sobrepeso|
|            Softball|                                   Sobrepeso|
|               None.|                                   Sobrepeso|
|              soccer|                                   Sobrepeso|
|                none|                                        NULL|
|                none|                                   Sobrepeso|
|        field hockey|                                        NULL|
|              soccer|                                        NULL|
|             Running|                                        NULL|
|Soccer and basket...|                                   Sobrepeso|
|intramural volley...|                          

In [23]:
resultdf4 = resultdf3.filter(resultdf3[1].isNotNull())

In [24]:
resultdf4.show()

+--------------------+--------------------------------------------+
|         type_sports|CASE WHEN (weight >= 150) THEN Sobrepeso END|
+--------------------+--------------------------------------------+
|          car racing|                                   Sobrepeso|
|         Basketball |                                   Sobrepeso|
|            Softball|                                   Sobrepeso|
|               None.|                                   Sobrepeso|
|              soccer|                                   Sobrepeso|
|                none|                                   Sobrepeso|
|Soccer and basket...|                                   Sobrepeso|
|              Hockey|                                   Sobrepeso|
|              Hockey|                                   Sobrepeso|
|              hockey|                                   Sobrepeso|
|          basketball|                                   Sobrepeso|
|              Soccer|                          

In [26]:
resultdf4.filter((resultdf3[0]!= 'none') & (resultdf3[0]!= 'nan') & (resultdf3[0]!= 'None') & (resultdf3[0]!= 'None.')).show()

+--------------------+--------------------------------------------+
|         type_sports|CASE WHEN (weight >= 150) THEN Sobrepeso END|
+--------------------+--------------------------------------------+
|          car racing|                                   Sobrepeso|
|         Basketball |                                   Sobrepeso|
|            Softball|                                   Sobrepeso|
|              soccer|                                   Sobrepeso|
|Soccer and basket...|                                   Sobrepeso|
|              Hockey|                                   Sobrepeso|
|              Hockey|                                   Sobrepeso|
|              hockey|                                   Sobrepeso|
|          basketball|                                   Sobrepeso|
|              Soccer|                                   Sobrepeso|
|              Tennis|                                   Sobrepeso|
|   tennis soccer gym|                          